## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_DataBase/WeatherPyDatabase.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Torbay,40,CA,scattered clouds,52,47.67,-52.73,71.01,8.05
1,1,Saint-Philippe,33,RE,scattered clouds,63,-21.36,55.77,66.20,6.93
2,2,Skelleftea,0,SE,clear sky,88,64.75,20.95,64.40,8.05
3,3,Cabudare,3,VE,light rain,51,10.03,-69.09,86.00,17.22
4,4,Cassilandia,39,BR,scattered clouds,35,-19.11,-51.73,73.53,7.11


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature for your vacation?"))
max_temp= float(input("What is the maximum tempterature for your vacation?"))


What is the minimum temperature for your vacation?75
What is the maximum tempterature for your vacation?90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df= city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,Cabudare,3,VE,light rain,51,10.03,-69.09,86.00,17.22
14,14,Nouadhibou,0,MR,clear sky,78,20.93,-17.03,75.20,17.22
16,16,Jamestown,75,US,broken clouds,47,42.10,-79.24,84.00,8.05
17,17,Mitsamiouli,42,KM,scattered clouds,72,-11.38,43.28,76.87,3.11
18,18,Dhidhdhoo,4,MV,clear sky,70,6.88,73.10,83.64,6.55


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Cloudiness             0
Country                0
Current Description    0
Humidity               0
Lat                    0
Lng                    0
Max Temp               0
Wind Speed             0
dtype: int64

In [12]:
preferred_cities_df.count()

City_ID                237
City                   237
Cloudiness             237
Country                237
Current Description    237
Humidity               237
Lat                    237
Lng                    237
Max Temp               237
Wind Speed             237
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cabudare,VE,86.00,light rain,10.03,-69.09,
14,Nouadhibou,MR,75.20,clear sky,20.93,-17.03,
16,Jamestown,US,84.00,broken clouds,42.10,-79.24,
17,Mitsamiouli,KM,76.87,scattered clouds,-11.38,43.28,
18,Dhidhdhoo,MV,83.64,clear sky,6.88,73.10,
19,Nantucket,US,77.00,clear sky,41.28,-70.10,
20,Methoni,GR,75.20,clear sky,36.82,21.70,
22,Kollam,IN,82.40,mist,8.88,76.60,
23,Djibo,BF,80.26,few clouds,14.10,-1.63,
26,Bardiyah,LY,76.10,clear sky,31.76,25.09,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for i, hotels in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat= hotels["Lat"]
    Lng= hotels["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]= f"{Lat}, {Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_url=requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"]= hotel_url["results"][0]["name"]
    except(IndexError):
        print("Skipping...")
        pass

Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...
Skipping...


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
hotel_df.isnull().count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [17]:
hotel_df.count()

City                   237
Country                237
Max Temp               237
Current Description    237
Lat                    237
Lng                    237
Hotel Name             237
dtype: int64

In [18]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [20]:
# 8a. Create the output File (CSV)
output_data_file= "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")



In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [25]:
# 11a. Add a marker layer for each city to the map. 
description=hotel_df["Current Description"]
fig= gmaps.figure(center=(30,31), zoom_level=1.5)
heat_layer= gmaps.heatmap_layer(locations, dissipating= False,
                               max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
plt.savefig("../Vacation_Search/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>